In [4]:
#Loading dataset
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('../data/loan-details.pdf')
docs=loader.load()

In [8]:
#Splitting document
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=20
)
texts = text_splitter.split_documents(docs)

In [11]:
#Importing Env variables
import os
from dotenv import load_dotenv,find_dotenv


dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [12]:
# Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(docs,OpenAIEmbeddings())

In [13]:
#Test - Similarity search
query = "5 types of loan schemes"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

Loans Description  
Smart bank mainl y has 5 type s of loan schemes.  
1. Overdra ft Facilities  
2. Housing Loans  
3. Education Loans  
4. Pensioners ’ Loans  
5. Personal Loans  
1. Over draft  Facilities  
An overdraft is the ideal way to manage your cash flow at competitive rates of interest at speed. It is 
linked to your current account and you can use as much as you need, up to your limit. You can also apply 
for and either increase or decrease your limit through Internet banking. The controls are in your hands.  
• An overdraft is quick and easy to arrange  
• The cash is available when you need it  
• You only pay interest on what you use, not on the full amount of your set limit  
• Sole proprietors can apply for an overdraft or increase their limits   
2. Housing Loans  
2.1. Housing Loan Sche me 
• Purchase a land to construct a house later  
• Purchase a land and to construct a house  
• Construct a house in a land owned by the borrower/s  
• Purchase a house/ partly cons

In [14]:
#Load Finetuned Model
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="ft:gpt-3.5-turbo-0125:personal:loansupport:9L2FZO36")

In [15]:
#Create Prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You're operating as a loan support assistant for a bank. Your task is to assist users with inquiries regarding various loan products and processes. Keep responses concise and relevant. If the information requested is not available, do not provide any details.

Question: {input}
Context:
{context}
""")


In [16]:
# Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
# Retriever
retriever = db.as_retriever()

In [18]:
# Retriever Chain
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [28]:
response=retrieval_chain.invoke({"input":"what do you know about areoplanes"})

In [29]:
response["answer"]

"I'm here to provide information on Smart Bank's loan schemes, including overdraft facilities, housing loans, education loans, pensioners' loans, and personal loans. If you have any specific questions about these schemes, feel free to ask."